In [1]:
!pip install konlpy

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import *
import pickle
from konlpy.tag import Okt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
pd.set_option('display.unicode.east_asian_width',True)

In [4]:
with open('/content/datasets/news_token.pickle', 'rb') as f:
    token = pickle.load(f)

In [5]:
with open('/content/datasets/category_encoder.pickle', 'rb') as f:
    encoder = pickle.load(f)
category = encoder.classes_
print(category)

['Culture' 'Economic' 'IT' 'Politics' 'Social' 'World']


In [6]:
df_news_today = pd.read_csv(
    '/content/datasets/naver_news_titles_210617_headline.csv',
    index_col=0)
print(df_news_today.head())
print(df_news_today.info())

                                                        title  category
0         합당 속도 공감대 속 주도권 기싸움 벌인               Politics
1     영상  이준석 안철수  전쟁 같은 합당 되지 않게            Politics
2   안철수 이준석 당 대표로서 첫 공식 회동 합당 논의           Politics
3  이준석 안철수  합당 의지 변함 없어   새 당명 엔 이견......  Politics
4   김기현 국민의힘 대표 국회 교섭단체 대표연설 LIVE ......    Politics
<class 'pandas.core.frame.DataFrame'>
Int64Index: 206 entries, 0 to 205
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     206 non-null    object
 1   category  206 non-null    object
dtypes: object(2)
memory usage: 4.8+ KB
None


In [7]:
df = df_news_today.drop_duplicates(subset=['title'])
sum_dup = df.title.duplicated().sum()
print(sum_dup)

0


In [8]:
df.reset_index(drop=True, inplace=True)

In [9]:
X = df['title']
Y = df['category']

In [10]:
# Okt를 활용한 형태소 분석
okt = Okt()
for i in range(len(X)):
    X[i] = okt.morphs(X[i])
print(X)

0      [합당, 속도, 공감, 대, 속, 주도, 권, 기, 싸움, 벌인]...          
1      [영상, 이준석, 안철수, 전쟁, 같은, 합당, 되지, 않게]...          
2      [안철수, 이준석, 당, 대표, 로서, 첫, 공식, 회, 동, 합당, 논의]...
3      [이준석, 안철수, 합당, 의지, 변함, 없어, 새, 당, 명, 엔, 이견]...
4      [김기현, 국민, 의, 힘, 대표, 국회, 교섭단체, 대표, 연설, LIVE]...
                                     ...                                
201    [지금, 은, 과학, 알츠하이머, 치매, 진행, 늦출, 수, 있다]...      
202    [유용하, 기자, 의, 사이언스, 톡, 생쥐, 를, 빼고, 과학, 연구, 를, 말,...
203    [유용하, 기자, 의, 사이언스, 톡, 수, 포자, 되면, 뇌, 인지력, 떨어져요]...
204    [유용하, 기자, 의, 사이언스, 톡, 엄마, 가, 아이, 구, 하려, 위험, 무릅...
205    [유용하, 기자, 의, 사이언스, 톡, 무작위, 같지만, 당신, 행동, 엔, 패턴,...
Name: title, Length: 206, dtype: object


In [11]:
#stopword
stopwords = pd.read_csv('/content/datasets/stopwords.csv')

In [12]:
# 스탑워드를 이용한 전처리
for i in range(len(X)):
    result = []
    for j in range(len(X[i])):
        if len(X[i][j]) > 1:
            if X[i][j] not in list(stopwords['stopword']):
                result.append(X[i][j])
    X[i] = ' '.join(result)
print(X)

0                        합당 속도 공감 주도 싸움 벌인
1               영상 이준석 안철수 전쟁 합당 되지 않게
2                    안철수 이준석 대표 공식 합당 논의
3                    이준석 안철수 합당 의지 변함 이견
4        김기현 국민 대표 국회 교섭단체 대표 연설 LIVE
                            ...                       
201                     과학 알츠하이머 치매 진행 늦출
202      유용하 기자 사이언스 생쥐 빼고 과학 연구 이유
203     유용하 기자 사이언스 포자 되면 인지력 떨어져요
204    유용하 기자 사이언스 엄마 하려 위험 무릅쓰 까닭
205       유용하 기자 사이언스 무작위 같지만 행동 패턴
Name: title, Length: 206, dtype: object


In [13]:
tokened_X = token.texts_to_sequences(X) # X로 숫자로 변환
print(tokened_X[0])

[877, 219, 1245, 576, 900, 9809]


In [14]:
# 패딩 / 빈공간 0으로 채워주기
X_pad = pad_sequences(tokened_X, 27)
print(X_pad[:10])

[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0   877   219  1245
    576   900  9809]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0    41     1   211   385
    877  2291  3264]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0   211     1    19
    451   877   168]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     1   211
    877  1491  4089]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     0   555     7    19   266  3733
     19  1139 15093]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     7   555    21  3733    19  1139  4114
   2672

In [20]:
model = load_model('/content/news_classification_0.740029513835907.h5')

In [21]:
predict = model.predict(X_pad)
print(predict[0])

[1.5599200e-01 1.7563527e-02 8.2413882e-01 2.2701597e-04 1.1707058e-03
 9.0795214e-04]


In [22]:
result = np.argmax(predict[0])

In [23]:
predict_category = []
for pred in predict:
    predict_category.append(category[np.argmax(pred)])

In [24]:
df_news_today['predict'] = predict_category
print(df_news_today.head())

                                                        title  ...   predict
0         합당 속도 공감대 속 주도권 기싸움 벌인               ...        IT
1     영상  이준석 안철수  전쟁 같은 합당 되지 않게            ...  Politics
2   안철수 이준석 당 대표로서 첫 공식 회동 합당 논의           ...  Politics
3  이준석 안철수  합당 의지 변함 없어   새 당명 엔 이견......  ...  Politics
4   김기현 국민의힘 대표 국회 교섭단체 대표연설 LIVE ......    ...  Politics

[5 rows x 3 columns]


In [25]:
df_news_today['OX'] = 0
print(df_news_today.head())

                                                        title  ... OX
0         합당 속도 공감대 속 주도권 기싸움 벌인               ...  0
1     영상  이준석 안철수  전쟁 같은 합당 되지 않게            ...  0
2   안철수 이준석 당 대표로서 첫 공식 회동 합당 논의           ...  0
3  이준석 안철수  합당 의지 변함 없어   새 당명 엔 이견......  ...  0
4   김기현 국민의힘 대표 국회 교섭단체 대표연설 LIVE ......    ...  0

[5 rows x 4 columns]


In [26]:
for i in range(len(df_news_today.predict)):
    if df_news_today.category[i] == df_news_today.predict[i]:
        df_news_today.OX[i] = 'O'
    else:
        df_news_today.OX[i] = 'X'
print(df_news_today.head())

                                                        title  ... OX
0         합당 속도 공감대 속 주도권 기싸움 벌인               ...  X
1     영상  이준석 안철수  전쟁 같은 합당 되지 않게            ...  O
2   안철수 이준석 당 대표로서 첫 공식 회동 합당 논의           ...  O
3  이준석 안철수  합당 의지 변함 없어   새 당명 엔 이견......  ...  O
4   김기현 국민의힘 대표 국회 교섭단체 대표연설 LIVE ......    ...  O

[5 rows x 4 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [27]:
 print(df_news_today.OX.value_counts() / len(df_news_today.OX))

O    0.538835
X    0.461165
Name: OX, dtype: float64


In [28]:
df_news_today.iloc[-30: , 0:3]

,title,category,predict
176,우주정거장 건설 선저우 12호 유인우주선 오늘 발사,World,World
177,중국 우주인 3명 자체 우주정거장에 보낸다 우주로 간다 ...,World,World
178,도쿄 아파트값 연봉의 11배인데 서울은 언론도 놀랐다 정영효의 인사이드재팬...,World,World
179,한 대당 5000만원 넘지만 서 코로나에 인기 폭발한 정영효의 인사이드 재팬...,World,World
180,스님 절반이 부업 잘나가던 불교 위기 맞은 까닭 정영효의 인사이드 재팬 ...,World,Economic
181,탈석탄사회 추진에 나무젓가락 가격 뛰는 이유 정영효의 인사이드 재팬 ...,World,Economic
182,산속 조난자 위치 소리 로 정확도 10배 높은 음원 위치 추적기술 나왔다...,IT,World
183,살려주세요 산 속 조난자 위치 소리로 찾는다,IT,Economic
184,살려주세요 산속 조난사고 소리로 정확하게 찾아낸다...,IT,Social
185,살려주세요 비명 위치 실시간으로 찾는다,IT,Culture
